In [11]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import files
import io
import pandas as pd
from google.colab import files
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [12]:
from google.colab import files

In [13]:
uploaded=files.upload()

Saving test_motion_data.csv to test_motion_data.csv


In [14]:
test_motion_data = list(uploaded.keys())[0]

In [15]:
train0 = pd.read_csv(io.BytesIO(uploaded[test_motion_data ]))

In [16]:
fig = make_subplots(specs=[[{"secondary_y": False}]])
fig.add_trace(go.Scatter(x=train0['Timestamp'], y=train0['AccX'], name='AccX'), secondary_y=False)
fig.update_layout(autosize=False, width=700, height=400, title_text="Train AccX")
fig.update_xaxes(title_text="Timestamp")
fig.update_yaxes(title_text="Motion", secondary_y=False)
fig.show()

In [18]:
train0=train0.sample(frac=1, random_state=2021)

In [19]:
display(train0[0:3])

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class,Timestamp
677,0.574698,1.505383,-1.435466,-0.036041,0.021609,0.074984,AGGRESSIVE,819288
2621,-0.351754,0.675662,0.837360,-0.085521,-0.054138,-0.025809,SLOW,820459
1955,1.202601,-0.570622,-1.269001,-0.010996,-0.041921,-0.078343,SLOW,820099


In [20]:
Name=sorted(train0['Class'].unique().tolist())
print(Name)
print(len(Name))
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N))
reverse_mapping=dict(zip(N,Name))
train0['Class']=train0['Class'].map(normal_mapping)
print(normal_mapping)

['AGGRESSIVE', 'NORMAL', 'SLOW']
3
{'AGGRESSIVE': 0, 'NORMAL': 1, 'SLOW': 2}


In [21]:
print(train0.columns.tolist())

['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'Class', 'Timestamp']


In [22]:
dummies =  pd.get_dummies(train0['Class'])
dummies.columns=Name
display(dummies)
train0=train0.drop(['Class'],axis=1)
train2=pd.concat([train0,dummies],axis=1)
display(train2)

,AGGRESSIVE,NORMAL,SLOW
677,True,False,False
2621,False,False,True
1955,False,False,True
2907,False,False,True
2465,False,False,True
...,...,...,...
2724,False,False,True
1704,False,True,False
1991,False,False,True
1425,False,True,False


,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Timestamp,AGGRESSIVE,NORMAL,SLOW
677,0.574698,1.505383,-1.435466,-0.036041,0.021609,0.074984,819288,True,False,False
2621,-0.351754,0.675662,0.837360,-0.085521,-0.054138,-0.025809,820459,False,False,True
1955,1.202601,-0.570622,-1.269001,-0.010996,-0.041921,-0.078343,820099,False,False,True
2907,-0.192239,0.904195,-1.030318,-0.032376,0.019777,-0.000764,820613,False,False,True
2465,-0.168332,0.504154,0.586774,0.010385,0.031994,-0.000153,820375,False,False,True
...,...,...,...,...,...,...,...,...,...,...
2724,-0.631274,-0.087172,0.754374,0.040928,0.043601,0.094531,820515,False,False,True
1704,-0.696055,0.231856,0.003220,0.032987,0.088194,0.007789,819921,False,True,False
1991,-1.061853,-0.666937,-1.371145,-0.079412,-0.135383,-0.033139,820119,False,False,True
1425,-1.322549,-0.560630,-0.188014,0.011606,-0.099342,-0.006261,819770,False,True,False


In [26]:
from sklearn.model_selection import train_test_split

train2, test2 = train_test_split(train2, test_size=0.2, random_state=42)  # Adjust test_size as needed

target = Name
trainY = train2[Name]
trainX = train2.drop(Name, axis=1)
testY = test2[Name]
testX = test2.drop(Name, axis=1)

In [27]:
df_columns = list(trainX.columns)
print(df_columns)

['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'Timestamp']


In [28]:
def create_numeric_feature(input_df):
    use_columns = df_columns
    return input_df[use_columns].copy()

In [29]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [30]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)

    return out_df

In [31]:
train_feat_df = to_feature(trainX)
test_feat_df = to_feature(testX)

100%|██████████| 1/1 [00:00<00:00, 340.89it/s]


createcreate_numeric_feature  0.001[s]


100%|██████████| 1/1 [00:00<00:00, 596.71it/s]

createcreate_numeric_feature  0.001[s]


In [32]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv,
             params: dict=None,
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)

        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train,
                    eval_set=[(x_valid, y_valid)],
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [33]:
params = {
    'objective': 'rmse',
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5,
    'n_estimators': 10000,
    'colsample_bytree': .5,
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain',
    'random_state': 71,
    'num_leaves': 62
}

In [34]:
y = trainY
print(y.shape)
print(type(y))

(2467, 3)
<class 'pandas.core.frame.DataFrame'>


In [35]:
ydf=pd.DataFrame(y)
ydf

,AGGRESSIVE,NORMAL,SLOW
1792,False,True,False
1572,False,True,False
667,True,False,False
1448,False,True,False
2162,False,False,True
...,...,...,...
326,True,False,False
1626,False,True,False
1634,False,True,False
342,True,False,False


In [37]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv,
             params: dict=None,
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    # Replaced np.float with np.float64
    oof_pred = np.zeros_like(y, dtype=np.float64)

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)

        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train,
                    eval_set=[(x_valid, y_valid)],
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [38]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df],
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df,
                  x='feature_importance',
                  y='column',
                  order=order,
                  ax=ax,
                  palette='viridis',
                  orient='h')

    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()

    return fig,ax

In [40]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv,
             params: dict=None,
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    # Replaced np.float with np.float64
    oof_pred = np.zeros_like(y, dtype=np.float64)

    for i, (idx_train, idx_valid) in enumerate(cv):
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)

        with Timer(prefix='fit fold={} '.format(i)):
            # Pass early_stopping_rounds to fit method within a eval_set parameter
            clf.fit(x_train, y_train,
                    eval_set=[(x_valid, y_valid)],
                    # The below line was corrected
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [43]:
TName=['YES','NO']
for i in range(len(Name)):
    try:
        PREDi=PREDS[i]
    except IndexError:
        print(f"Warning: PREDS does not have an element at index {i}. Skipping this class.")
        continue # Skip to the next class if PREDS is too short
    PREDi2=[]
    for item in PREDi:
        PREDi2+=[int(item)]

    ANSi=testY.iloc[:,i]
    ANSi2=[]
    for item in ANSi:
        ANSi2+=[item]

    print(Name[i])
    print(classification_report(np.array(ANSi2), np.array(PREDi2), target_names=TName, digits=4))